<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/team8_lab04/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Movies-Activity").getOrCreate()

df_movies = spark.read \
            .option("inferSchema", "true") \
            .json("datasets/movies.json")

In [5]:
df_movies.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                NULL|             NULL|      Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land Girls|        NULL|  146083|     

In [6]:
df_solution_1 = df_movies.withColumn('ProfitMargin',
 (df_movies['Worldwide_Gross']-df_movies['Production_Budget']) / df_movies['Production_Budget'])

In [7]:
df_solution_1.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|       ProfitMargin|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|                NULL|             NULL|      Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|        

In [8]:
from pyspark.sql.functions import when
df_solution_2 = df_movies.withColumn('Box_Office_Hit', when(df_movies['Worldwide_Gross'] > 2 * df_movies['Production_Budget'], "Hit").otherwise("Flop"))

In [9]:
df_solution_2.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+--------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Box_Office_Hit|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+--------------+
|                NULL|             NULL|      Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|   

In [10]:
from pyspark.sql.functions import when, col, to_date, year
df_movies = df_movies.withColumn("Release_Date_Formatted", 
    when(col("Release_Date").rlike(r'^[0-9]{2}-[A-Za-z]{3}-[0-9]{2}$'), to_date(col("Release_Date"), "dd-MMM-yy"))
    .when(col("Release_Date").rlike(r'^[0-9]{1}-[A-Za-z]{3}-[0-9]{2}$'), to_date(col("Release_Date"), "d-MMM-yy"))
    .when(col("Release_Date").rlike(r'^\d{4}-\d{2}-\d{2}$'), to_date(col("Release_Date"), "yyyy-MM-dd"))
    .when(col("Release_Date").rlike(r'^\d{2}/\d{2}/\d{2}$'), to_date(col("Release_Date"), "MM/dd/yy"))
    .otherwise(None)
)

df_movies = df_movies.withColumn("Release_Year", 
    when(year(col("Release_Date_Formatted")) > 2000, year(col("Release_Date_Formatted")) - 100)
    .otherwise(year(col("Release_Date_Formatted")))
)

df_movies.select("Title", "Release_Date", "Release_Date_Formatted", "Release_Year").show(truncate=False)

+------------------------------+------------+----------------------+------------+
|Title                         |Release_Date|Release_Date_Formatted|Release_Year|
+------------------------------+------------+----------------------+------------+
|The Land Girls                |12-Jun-98   |2098-06-12            |1998        |
|First Love, Last Rites        |7-Aug-98    |2098-08-07            |1998        |
|I Married a Strange Person    |28-Aug-98   |2098-08-28            |1998        |
|Let's Talk About Sex          |11-Sep-98   |2098-09-11            |1998        |
|Slam                          |9-Oct-98    |2098-10-09            |1998        |
|Mississippi Mermaid           |15-Jan-99   |2099-01-15            |1999        |
|Following                     |4-Apr-99    |2099-04-04            |1999        |
|Foolish                       |9-Apr-99    |2099-04-09            |1999        |
|Pirates                       |1-Jul-86    |2086-07-01            |1986        |
|Duel in the Sun

In [11]:
from pyspark.sql.functions import when
import pyspark.sql.functions as F 
# Number 4
df_4 = df_movies.withColumn("IMDB_Rating_Category",
                            when(df_movies["IMDB_Rating"] >= 7.0, "High")\
                            .when(df_movies["IMDB_Rating"] < 5.0, "Low")\
                            .otherwise("Medium"))

df_4.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------------+------------+--------------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Release_Date_Formatted|Release_Year|IMDB_Rating_Category|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+----------------------+------------+--------------------+
|                NULL|             NULL|      Gramercy|        6.1|   

In [13]:
import pyspark.sql.functions as F 
# Number 5
df_5 = df_movies.groupBy("Distributor").agg(F.avg("IMDB_Rating"))

df_5.show()

+--------------------+------------------+
|         Distributor|  avg(IMDB_Rating)|
+--------------------+------------------+
|Oscilloscope Pict...|               6.2|
|               Savoy| 6.533333333333334|
|             Embassy|              NULL|
|         Fader Films|               6.5|
|       October Films|               6.4|
|              Strand| 6.322222222222223|
|             Trimark| 5.085714285714286|
|              Matson|               7.3|
|      Cinema Service|               7.1|
| Weinstein/Dimension|5.7250000000000005|
|Providence Entert...|               3.3|
|      Big Fat Movies|               3.2|
|     Newmarket Films|7.4142857142857155|
|    First Run/Icarus|              NULL|
|  Cloud Ten Pictures|               5.0|
|   Outrider Pictures|               7.0|
|     Slowhand Cinema|               5.3|
|   Paramount Vantage|               7.2|
|   Magnolia Pictures|            6.4625|
|  Kino International|6.0200000000000005|
+--------------------+------------